In [ ]:
import gc


In [ ]:
# Clear GPU cache
torch.cuda.empty_cache()


In [ ]:

# Trigger garbage collection
gc.collect()


0

# Install packages

In [ ]:
!pip install git+https://github.com/huggingface/accelerate


  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-k46e2emc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-k46e2emc
  Resolved https://github.com/huggingface/accelerate to commit cd7df4117d92f660965d5f737364395b5693a535
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -q peft bitsandbytes wandb

In [ ]:
!pip install scipy

In [ ]:
!apt-get update && apt-get install -y git

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:3 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [32.5 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4069 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1200 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1496 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [3686 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [29.8 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3536 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3594 kB]
Fetched 17.9 MB in 22s (804

In [ ]:
!pip install git+https://github.com/huggingface/trl.git@7630f877f91c556d9e5a3baa4b6e2894d90ff84c


  Cloning https://github.com/huggingface/trl.git (to revision 7630f877f91c556d9e5a3baa4b6e2894d90ff84c) to /tmp/pip-req-build-b39bg5s1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-b39bg5s1
  Running command git rev-parse -q --verify 'sha^7630f877f91c556d9e5a3baa4b6e2894d90ff84c'
  Running command git fetch -q https://github.com/huggingface/trl.git 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Running command git checkout -q 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Resolved https://github.com/huggingface/trl.git to commit 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Import packages

In [ ]:
# Import necessary packages for the fine-tuning process
import os
import torch
from datasets import load_dataset,DatasetDict

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,                     # Creating pipelines for model inference
    logging,                      # Logging information during training
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datetime import datetime

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Authenticate with wandb

In [ ]:
import wandb
#create account and input ur key here
wandb.login(key='WANDB API KEY')

wandb: Currently logged in as: bertha-tam (298bwanderchat). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Authenticate with HF

In [ ]:
import huggingface_hub
huggingface_hub.login(token='PASS YOUR HF TOKEN KEY')

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!huggingface-cli whoami

sherrys


# Initialize model and dataset by name

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# if you have ur own custom dataset, follow this guide to publish it https://huggingface.co/docs/datasets/en/upload_dataset
dataset_name = "beraht/Reddit_RAFT_Falcon"

# Fine-tuned model name
new_model = "426_mistral_RAFT_50e_10s" # This is model name uploaded on HuggingFace with user sherrys, feel free to download and explore more, if you have your own trained model put the name of that model here.

# Define LLM model parameters

In [ ]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [ ]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

In [ ]:
# Output directory where the model predictions and checkpoints will be stored
output_dir = "Save the checkpoints"

In [ ]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# Load dataset and train/val/test split

In [ ]:
# Step 1 : Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

In [ ]:
train_test_dataset = dataset.train_test_split(test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = train_test_dataset['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_test_dataset['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [ ]:
train_test_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction'],
        num_rows: 203
    })
    test: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction'],
        num_rows: 12
    })
    valid: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction'],
        num_rows: 11
    })
})

In [ ]:
path_saved="./426_Mistral_RAFT_datasplit"
import pickle
with open(path_saved+"train_test_dataset.pkl","wb") as f:
    pickle.dump(train_test_dataset,f)

with open(path_saved+"test_valid_dataset.pkl","wb") as f:
    pickle.dump(test_valid,f)

with open(path_saved+"train_test_valid_dataset.pkl","wb") as f:
    pickle.dump(train_test_valid_dataset,f)


In [ ]:
import pickle

# Load train_test_dataset
with open("426_Mistral_RAFT_datasplittrain_test_dataset.pkl", "rb") as f:
    train_test_dataset = pickle.load(f)

# Load test_valid split
with open("426_Mistral_RAFT_datasplittest_valid_dataset.pkl", "rb") as f:
    test_valid = pickle.load(f)

# Load train_test_valid split
with open("426_Mistral_RAFT_datasplittrain_test_valid_dataset.pkl", "rb") as f:
    train_test_valid_dataset = pickle.load(f)



In [ ]:
def transform_data(example):
    ques = example['instruction']
    ans = example['cot_answer']
    return {
        "text": f"<s>[INST] {ques} [/INST] {ans} </s>"
    }

transformed_dataset = train_test_valid_dataset.map(transform_data)
transformed_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'text'],
        num_rows: 203
    })
    test: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'text'],
        num_rows: 12
    })
    valid: Dataset({
        features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'text'],
        num_rows: 11
    })
})

In [ ]:
transformed_dataset['train']['text'][1]

"<s>[INST] <DOCUMENT>Good hotel ok. I would go back to Patong, but prefer Angeles in PI . The biggest jokes are baht bus prices in parting and the price for the copy goods trainers jeans etc.I have not seen snow like this in Brussels in almost years . If youre here between June and September you might also see the sun come out for a day or two .</DOCUMENT>\n<DOCUMENT>The biggest factor for me is ensuring wherever I book, I get a good cancellation policy . There is no reason for Expedia or Booking to offer better rates than the hotels itself .I have always used a physical sim when Ive traveled in the EU but Ive heard esims are good as well . I have an airalo code for off if anyone is interested . If you plan to country hop a bunch, youll be buying sims at every place .The most wild backpacking trip Ive been on. I went around Mt Eielson two years ago .</DOCUMENT>\n<DOCUMENT>Yes Can I take my emotional support duck Santiago as well? If its a rubber duck Yes., Vote for Santiago., After som

# Define training procedures

In [ ]:
# Step 2 :Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
# Step 3 :Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [ ]:
# Step 4 :Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.26s/it]


In [ ]:
# Step 5 :Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Step 6 :Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# Step 6 :Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
project = "raft_falcon_50e_10s"
base_model_name = "mistral"
#run_name = base_model_name + "-" + project

# Number of training epochs
num_train_epochs = 50 # used to be 50

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2    #4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 2     #4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

evaluation_strategy = "steps"

eval_steps = 10

# Save checkpoint every X updates steps
save_steps = 10

# Log every X updates steps
logging_steps = 10

In [ ]:
# Step 7 :Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    evaluation_strategy = evaluation_strategy,
    eval_steps=eval_steps,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    report_to="wandb"
)

In [ ]:
from transformers import EarlyStoppingCallback
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset['train'],
    eval_dataset=transformed_dataset['valid'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:228: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


# Train

In [ ]:
# Step 9 :Train model
trainer.train()

# Step 10 :Save trained model
trainer.model.save_pretrained(new_model)

Step,Training Loss,Validation Loss
10,2.537400,1.935675
20,1.591700,1.130509
30,0.933700,0.510545


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
new_model

In [ ]:
model.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)


to download the trained RAFT model and use it further , feel free to download from
https://huggingface.co/sherrys/426_mistral_RAFT_50e_10s  

Section 1: Parameters to tune
Load a mistralai/Mistral-7B-Instruct-v0.2 model and trained it on the
beraht/Reddit_RAFT_Falcon dataset
Feel free to use a different dataset.

Section 2: QLoRA parameters we can play around with many other parameters around , but due to limited harware and memory , we could not experiment extensively to  tune the hyperparameters.
See this article for more information about LoRA parameters.
The Mistral-Instruct-7b-v0.2 model loaded directly in 4-bit precision using the NF4 type.

Section 3: Other parameters
To get more information about the other parameters, check the TrainingArguments, PeftModel, and SFTTrainer documentation.
